retain possession extras

In [ ]:
# import libraries
from statsbombpy import sb
import json
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer.pitch import Pitch
import seaborn as sns
import os
import re

In [ ]:
# obtain list of matches for specific competition and season 
df_matches = sb.matches(competition_id=55, season_id=43)
df_matches

In [ ]:
df_throwIn_new_index = df_throwIn.reset_index()
df_throwIn_final = df_throwIn_new_index.rename(columns={'level_0': 'full_index','Unnamed: 0': 'old_index'})
df_throwIn_final

In [ ]:
# create list of all competitions to analyze

directory = 'data/three-sixty'

# list of filenames
file_list = []
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        f_number = re.split("[/.]", f)[2]
        file_list.append(f_number)

file_list

In [ ]:
# create dictionary deeming each throw in a success or failure
# success defined as retaining possession for at least 7 seconds 

from datetime import datetime

throw_in_outcomes = {}

current_possession = 0
i = 0

throwin_count = 0

for match in file_list:
    throwin_row = 0
    looking_for_result = False
    elapsed_time = 0
    current_possession = 0
    i = 0

    initial_match_df = df_throwIn_final[df_throwIn_final["match_id"]== int(match)]
    match_df = initial_match_df.reset_index()

    while i < len(match_df):
        if (match_df.loc[i, "pass_type"] == "Throw-in") and (looking_for_result == False):
            throwin_count += 1
            throwin_row = i
            elapsed_time = 0
            start_time = match_df.loc[i, "timestamp"]
            start_dt_time = datetime.strptime(start_time, '%H:%M:%S.%f')
            current_possession = match_df.loc[i, "possession"]
            looking_for_result = True
            # print(throwin_row)
        
        elif looking_for_result == True:
            end_time = match_df.loc[i, "timestamp"]
            end_dt_time = datetime.strptime(end_time, '%H:%M:%S.%f')
            elapsed_time = elapsed_time + (end_dt_time - start_dt_time).total_seconds()
            start_dt_time = end_dt_time

            if elapsed_time >= 7:
                if match_df.loc[i, "possession"] == current_possession:
                    throw_in_outcomes[throwin_count] = [match_df.loc[throwin_row, "location"], "Success"]
                else:
                    throw_in_outcomes[throwin_count] = [match_df.loc[throwin_row, "location"], "Failure"]
                    i = throwin_row
                looking_for_result = False
        
        i = i + 1

throw_in_outcomes




In [ ]:
# linear regression
import numpy as np
from sklearn.linear_model import LinearRegression

list_of_x_coordinates = []
for throwin in throw_in_outcomes:
    coords = throw_in_outcomes[throwin][0]
    x_coord = coords.split(",")[0][1:]
    list_of_x_coordinates.append(float(x_coord))

list_of_outcomes = []
for throwin in throw_in_outcomes:
    outcome = throw_in_outcomes[throwin][1]
    if outcome == "Success":
        outcome_val = 1
    else:
        outcome_val = 0

    list_of_outcomes.append(outcome_val)

x = np.array(list_of_x_coordinates).reshape((-1, 1))
print(x)

y = np.array(list_of_outcomes)
print(y)

In [ ]:
# modeling

model = LinearRegression()

model.fit(x, y)

In [ ]:
r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")
print(f"intercept: {model.intercept_}")
print(f"slope: {model.coef_}")

In [ ]:
y_pred = model.predict(x)
print(f"predicted response:\n{y_pred}")

That didn't work... try with only data from half the field (don't think linear shape is working, maybe just using half the field would yield more of a linear trend?)

In [ ]:
list_of_x_coordinates = []
list_of_outcomes = []

for throwin in throw_in_outcomes:
    coords = throw_in_outcomes[throwin][0]
    x_coord = coords.split(",")[0][1:]
    if float(x_coord) <= 60:
        list_of_x_coordinates.append(float(x_coord))
        outcome = throw_in_outcomes[throwin][1]
        if outcome == "Success":
            outcome_val = 1
        else:
            outcome_val = 0
        list_of_outcomes.append(outcome_val)

x = np.array(list_of_x_coordinates).reshape((-1, 1))
print(x)

y = np.array(list_of_outcomes)
print(y)

In [ ]:
# new model

new_model = LinearRegression()

new_model.fit(x, y)

In [ ]:
r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")
print(f"intercept: {model.intercept_}")
print(f"slope: {model.coef_}")

In [ ]:
y_pred = model.predict(x)
print(f"predicted response:\n{y_pred}")

In [ ]:
# with statsmodels
x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)

In [ ]:
# create dictionary deeming each throw in a success or failure and including other info
# success defined as retaining possession for at least 7 seconds 

from datetime import datetime

throw_in_outcomes_extended = {}

current_possession = 0
i = 0

throwin_count = 0

for match in file_list:
    throwin_row = 0
    looking_for_result = False
    elapsed_time = 0
    current_possession = 0
    i = 0

    initial_match_df = df_throwIn_final[df_throwIn_final["match_id"]== int(match)]
    match_df = initial_match_df.reset_index()

    while i < len(match_df):
        if (match_df.loc[i, "pass_type"] == "Throw-in") and (looking_for_result == False):
            throwin_count += 1
            throwin_row = i
            elapsed_time = 0
            start_time = match_df.loc[i, "timestamp"]
            start_dt_time = datetime.strptime(start_time, '%H:%M:%S.%f')
            current_possession = match_df.loc[i, "possession"]
            looking_for_result = True
            # print(throwin_row)
        
        elif looking_for_result == True:
            end_time = match_df.loc[i, "timestamp"]
            end_dt_time = datetime.strptime(end_time, '%H:%M:%S.%f')
            elapsed_time = elapsed_time + (end_dt_time - start_dt_time).total_seconds()
            start_dt_time = end_dt_time

            if elapsed_time >= 7:
                
                if match_df.loc[i, "possession"] == current_possession:
                    throw_in_outcomes_extended[throwin_count] = [match_df.loc[throwin_row, "location"], match_df.loc[throwin_row, "pass_height"], match_df.loc[throwin_row, "pass_angle"], match_df.loc[throwin_row, "pass_length"], "Success"]
                
                else:
                    throw_in_outcomes_extended[throwin_count] = [match_df.loc[throwin_row, "location"], match_df.loc[throwin_row, "pass_height"], match_df.loc[throwin_row, "pass_angle"], match_df.loc[throwin_row, "pass_length"], "Failure"]
                    
                    i = throwin_row
                
                looking_for_result = False
        
        i = i + 1

throw_in_outcomes_extended

In [ ]:
full_list = []

list_of_x_coordinates = []

for throwin in throw_in_outcomes_extended:
    coords = throw_in_outcomes_extended[throwin][0]
    x_coord = coords.split(",")[0][1:]
    list_of_x_coordinates.append(float(x_coord))
    if throw_in_outcomes_extended[throwin][1] == "Low Pass":
        list_of_x_coordinates.append(0)
    else:
        list_of_x_coordinates.append(1)
    list_of_x_coordinates += (throw_in_outcomes_extended[throwin][2:4])
    full_list.append(list_of_x_coordinates)
    list_of_x_coordinates = []


list_of_outcomes = []
for throwin in throw_in_outcomes_extended:
    outcome = throw_in_outcomes_extended[throwin][4]
    if outcome == "Success":
        outcome_val = 1
    else:
        outcome_val = 0
    list_of_outcomes.append(outcome_val)

print(full_list)

In [ ]:
details_df = pd.DataFrame(full_list, columns = ['location', 'height', 'angle', 'distance'])

details_df

In [ ]:
x = np.asarray(details_df)
x

In [ ]:
y = np.array(list_of_outcomes)
print(y)

In [ ]:
import statsmodels.api as sm
model = sm.OLS(y, x)

In [ ]:
print(x.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [ ]:
model = LinearRegression()
model.fit(X_train, Y_train)

In [ ]:
r_sq = model.score(x,y)
print(r_sq)

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(x, y)

In [ ]:
print(regr.coef_)

In [ ]:
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

In [ ]:
# with statsmodels
x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)

In [ ]:
# how many successful throw-ins?

throwin_total = 0
successful_total = 0

for throwin in throw_in_outcomes:
    throwin_total += 1
    outcome = throw_in_outcomes[throwin][1]
    if outcome == "Success":
        successful_total += 1

success_percentage = successful_total/throwin_total

print("Overall success rate: " + str(success_percentage))


In [ ]:
# Throw-ins from which part of the field lead to the most success?

# split location into 4: penalty area (18 yards) on both sides and down the middle 
# area 1 (0-18 and 102-120): penalty area, area 2 (18-102) attacking area

penalty_total = 0
middle_total = 0

penalty_successes = 0
middle_successes = 0

for throwin in throw_in_outcomes:
    coords = throw_in_outcomes[throwin][0]
    x_coord = float(coords.split(",")[0][1:])
    outcome = throw_in_outcomes[throwin][1]

    if x_coord >= 102 or x_coord <= 18:
        middle_total += 1
        if outcome == "Success":
            middle_successes += 1
    
    else:
        penalty_total += 1
        if outcome == "Success":
            penalty_successes += 1

penalty_success_rate = penalty_successes/penalty_total

middle_success_rate = middle_successes/middle_total

print("Penalty area success rate: " + str(penalty_successes) + "/" + str(penalty_total) + " = " + str(penalty_success_rate))
print("Attacking area success rate: "  + str(middle_successes) + "/" + str(middle_total) + " = " +  str(middle_success_rate))

In [ ]:
from prettytable import PrettyTable

myTable = PrettyTable(["Throw-in Location", "Successes", "Total", "Success Rate"])

myTable.add_row(["Penalty", penalty_successes, penalty_total, round(penalty_success_rate, 4)])
myTable.add_row(["Attacking", middle_successes, middle_total, round(middle_success_rate, 4)])

print(myTable)

In [ ]:
# create dictionary deeming each throw in a success or failure and including which team threw in the ball
# success defined as retaining possession for at least 7 seconds 

from datetime import datetime

throw_in_outcomes_team = {}

current_possession = 0
i = 0

throwin_count = 0

for match in file_list:
    throwin_row = 0
    looking_for_result = False
    elapsed_time = 0
    current_possession = 0
    i = 0

    initial_match_df = df_throwIn_final[df_throwIn_final["match_id"]== int(match)]
    match_df = initial_match_df.reset_index()

    while i < len(match_df):
        if (match_df.loc[i, "pass_type"] == "Throw-in") and (looking_for_result == False):
            throwin_count += 1
            throwin_row = i
            elapsed_time = 0
            start_time = match_df.loc[i, "timestamp"]
            start_dt_time = datetime.strptime(start_time, '%H:%M:%S.%f')
            current_possession = match_df.loc[i, "possession"]
            looking_for_result = True
            # print(throwin_row)
        
        elif looking_for_result == True:
            end_time = match_df.loc[i, "timestamp"]
            end_dt_time = datetime.strptime(end_time, '%H:%M:%S.%f')
            elapsed_time = elapsed_time + (end_dt_time - start_dt_time).total_seconds()
            start_dt_time = end_dt_time

            if elapsed_time >= 7:
                
                if match_df.loc[i, "possession"] == current_possession:
                    throw_in_outcomes_team[throwin_count] = [match_df.loc[throwin_row, "possession_team"], "Success"]
                
                else:
                    throw_in_outcomes_team[throwin_count] = [match_df.loc[throwin_row, "possession_team"], "Failure"]
                    
                    i = throwin_row
                
                looking_for_result = False
        
        i = i + 1

throw_in_outcomes_team

In [ ]:
team_dict = {}

for attempt in throw_in_outcomes_team.values():
    team_name = attempt[0]
    
    if attempt[0] not in team_dict:
        if attempt[1] == "Success":
            team_dict[team_name] = [1, 0]
        else:
            team_dict[team_name] = [0, 1]
    
    else:
        if attempt[1] == "Success":
            team_dict[team_name][0] += 1
        else:
            team_dict[team_name][1] += 1

team_dict

In [ ]:
team_dict_updated = {}

for team in team_dict.keys():
    team_dict_updated[team] = (team_dict[team][0] / (team_dict[team][0] + team_dict[team][1]))

team_dict_updated

In [ ]:
from prettytable import PrettyTable

team_table = PrettyTable(["Team", "Possession retention success rate"])

for team in team_dict_updated.keys():
    team_table.add_row([team, round(team_dict_updated[team], 4)])

print(team_table.get_string(sortby="Possession retention success rate", reversesort=True))